## RGPD ET REGLEMENTATION

Je respecte les règles d’accès publiées par le site (conditions/mentions, et robots.txt quand accessible).

Je limite la charge : faible fréquence, temporisation, pas d’exploration infinie, cache local.

Je ne contourne aucune protection (authentification, CAPTCHA, paywall, restrictions techniques).

Je minimise la collecte : uniquement les champs nécessaires à l’objectif (agenda / tourisme).

Je n’ingère pas de données personnelles (et je supprime tout élément accidentellement collecté).

J’identifie clairement mon robot (User-Agent explicite, contact).

J’arrête immédiatement en cas de blocage explicite, demandes de retrait, ou signes de surcharge.

Avant historisation, j’anonymise les liens : remplacement de bordeaux-tourisme.com par tourisme.example.

(Le site publie des pages “Mentions légales” et “Politique de confidentialité / RGPD” utiles pour cadrer ces règles.)
bordeaux-tourisme.com


## Anonymisation de l'url via un .env puis .gitignore

In [35]:
from pathlib import Path
import os
from dotenv import load_dotenv

RACINE_PROJET = Path(r"C:\Users\HP-X360-1030-G3\PycharmProjects\notebooks\exercice2")

DATA = RACINE_PROJET / "data"
RAW = DATA / "raw"
PARQUET = DATA / "parquet"

RAW.mkdir(parents=True, exist_ok=True)
PARQUET.mkdir(parents=True, exist_ok=True)

dotenv_path = RACINE_PROJET / ".env"
load_dotenv(dotenv_path=dotenv_path)

DOMAINE_REEL = os.getenv("TOURISME_DOMAINE_REEL")
DOMAINE_STOCKAGE = os.getenv("TOURISME_DOMAINE_STOCKAGE")

if not DOMAINE_REEL or not DOMAINE_STOCKAGE:
    raise ValueError("Vérifiez votre .env : TOURISME_DOMAINE_REEL et TOURISME_DOMAINE_STOCKAGE doivent être définis.")

def anonymiser_url(url: str) -> str:
    return url.replace(DOMAINE_REEL, DOMAINE_STOCKAGE)

print("Configuration OK. Domaine stockage =", DOMAINE_STOCKAGE)

Configuration OK. Domaine stockage = tourisme.example


## Import des Dépendances

In [36]:
import sys
!{sys.executable} -m pip install --upgrade pip


## PARAMETRES + TEMPORISATION

In [37]:
import time
import random
import os

BASE_URL_REEL = f"https://{DOMAINE_REEL}"

USER_AGENT = os.getenv(
    "TOURISME_USER_AGENT",
    "Mozilla/5.0 (compatible; AtelierScraping/1.0; contact=you@example.com)"
)

MIN_SLEEP = float(os.getenv("TOURISME_MIN_SLEEP", "1.0"))
MAX_SLEEP = float(os.getenv("TOURISME_MAX_SLEEP", "2.5"))

def sleep_poli():
    time.sleep(random.uniform(MIN_SLEEP, MAX_SLEEP))


## robots.txt + session + GET HTML

In [38]:
import requests
from urllib.robotparser import RobotFileParser

def robots_autorise(url: str) -> bool:
    rp = RobotFileParser()
    try:
        rp.set_url(f"{BASE_URL_REEL}/robots.txt")
        rp.read()
        return rp.can_fetch(USER_AGENT, url)
    except Exception:
        # posture prudente : autoriser mais rester très limité
        return True

session = requests.Session()
session.headers.update({"User-Agent": USER_AGENT})

def get_html(url: str) -> str:
    if not robots_autorise(url):
        raise PermissionError("Accès refusé par robots.txt (URL non affichée).")
    r = session.get(url, timeout=30)
    r.raise_for_status()
    sleep_poli()
    return r.text


## Imports + URL “agenda”

In [39]:
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
from datetime import datetime
import re

AGENDA_LIST_URL = f"{BASE_URL_REEL}/agenda.html"


## URL + téléchargement + contrôle basique

In [40]:
AGENDA_LIST_URL = f"{BASE_URL_REEL}/agenda.html"

html = get_html(AGENDA_LIST_URL)

print("HTML récupéré (taille) :", len(html))
print("Début HTML (aperçu) :", html[:200])


HTML récupéré (taille) : 148583
Début HTML (aperçu) : <!DOCTYPE html>
<html lang="fr" dir="ltr" prefix="og: https://ogp.me/ns#">
  <head>
    <meta charset="utf-8" />
<meta name="description" content="Consultez tous les événements dans l&#039;agenda de B


## Parser HTML + compter les liens “candidats”

In [41]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, "lxml")

liens = soup.select("a[href]")
print("Nombre total de liens <a href> :", len(liens))

candidats = []
for a in liens:
    href = (a.get("href") or "").strip()
    texte = " ".join(a.get_text(" ", strip=True).split())

    if not href:
        continue
    if not href.lower().endswith(".html"):
        continue
    if "agenda.html" in href.lower():
        continue
    if ("/evenements/" not in href.lower()) and ("/agenda/" not in href.lower()):
        continue
    if not texte:
        continue

    candidats.append((texte, href))

print("Candidats (liens fiche) :", len(candidats))

# Aperçu contrôlé (pas d'URL absolue réelle)
for i, (texte, href) in enumerate(candidats[:10], start=1):
    print(i, "|", texte[:80], "|", href)


Nombre total de liens <a href> : 256
Candidats (liens fiche) : 76
1 | Ce week-end | https://www.bordeaux-tourisme.com/agenda/week-end.html
2 | Incontournables | https://www.bordeaux-tourisme.com/agenda/incontournables.html
3 | Avec les enfants | https://www.bordeaux-tourisme.com/agenda/sorties-enfants.html
4 | Concerts et spectacles | https://www.bordeaux-tourisme.com/agenda/concerts-spectacles.html
5 | Expositions | https://www.bordeaux-tourisme.com/agenda/expositions.html
6 | Fêtes & festivals | https://www.bordeaux-tourisme.com/agenda/fetes-festivals.html
7 | expositions | /agenda/expositions.html
8 | festivals | /agenda/fetes-festivals.html
9 | Avec les enfants | /agenda/sorties-enfants.html
10 | La grande roue de Noël de Bordeaux Du 18 décembre au 04 janvier | https://www.bordeaux-tourisme.com/evenements/grande-roue-noel-bordeaux.html


## Imports restants

In [42]:
from urllib.parse import urljoin
import re
import pandas as pd
from datetime import datetime


## Fonction d’extraction

In [43]:
def extraire_items_depuis_liste(html: str, url_source: str) -> list[dict]:
    soup = BeautifulSoup(html, "lxml")
    items = []

    for a in soup.select("a[href]"):
        href = (a.get("href") or "").strip()
        if not href:
            continue

        texte = " ".join(a.get_text(" ", strip=True).split())
        if not texte:
            continue

        href_lower = href.lower()
        if not href_lower.endswith(".html"):
            continue
        if "agenda.html" in href_lower:
            continue
        if ("/evenements/" not in href_lower) and ("/agenda/" not in href_lower):
            continue

        # Filtre "date" (on le garde pour commencer, mais on pourra l'assouplir si ça renvoie 0)
        if (" Du " not in f" {texte} ") and (" Le " not in f" {texte} "):
            continue

        url_abs = urljoin(url_source, href)

        m = re.match(r"^(.*?)(\s+(Du|Le)\s+.*)$", texte)
        titre = m.group(1).strip() if m else texte
        date_raw = m.group(2).strip() if m else None

        items.append({
            "scraped_at": datetime.now().isoformat(timespec="seconds"),
            "source_list_url": anonymiser_url(url_source),
            "title_raw": texte,
            "title": titre,
            "date_raw": date_raw,
            "_detail_url_fetch": url_abs,           # réel (mémoire uniquement)
            "detail_url": anonymiser_url(url_abs),  # anonymisé (stockable)
        })

    return list({it["_detail_url_fetch"]: it for it in items}.values())


## Exécution + DataFrame

In [44]:
items = extraire_items_depuis_liste(html, AGENDA_LIST_URL)
df_liste = pd.DataFrame(items)

print("Items retenus :", len(df_liste))

# Affichage sans la colonne réelle
df_liste.drop(columns=["_detail_url_fetch"], errors="ignore").head(10)


Items retenus : 60


,scraped_at,source_list_url,title_raw,title,date_raw,detail_url
0,2025-12-18T10:35:57,https://tourisme.example/agenda.html,La grande roue de Noël de Bordeaux Du 18 décem...,La grande roue de Noël de Bordeaux,Du 18 décembre au 04 janvier,https://tourisme.example/evenements/grande-rou...
1,2025-12-18T10:35:57,https://tourisme.example/agenda.html,La patinoire de Noël à Bordeaux Du 18 décembre...,La patinoire de Noël à Bordeaux,Du 18 décembre au 04 janvier,https://tourisme.example/evenements/patinoire-...
2,2025-12-18T10:35:57,https://tourisme.example/agenda.html,Marché de Noël de Bordeaux Du 18 décembre au 2...,Marché de Noël de Bordeaux,Du 18 décembre au 28 décembre,https://tourisme.example/evenements/marche-noe...
3,2025-12-18T10:35:57,https://tourisme.example/agenda.html,Bienvenue chez les Préhistos ! Du 18 décembre ...,Bienvenue chez les Préhistos !,Du 18 décembre au 22 mars,https://tourisme.example/evenements/bienvenue-...
4,2025-12-18T10:35:57,https://tourisme.example/agenda.html,"""Lune"", la nouvelle grande exposition de Cap S...","""Lune"", la nouvelle grande exposition de Cap S...",Du 18 décembre au 31 août,https://tourisme.example/evenements/lune-nouve...
5,2025-12-18T10:35:57,https://tourisme.example/agenda.html,"Exposition en réalité virtuelle, Carcassonne 1...","Exposition en réalité virtuelle, Carcassonne 1304",Du 18 décembre au 17 janvier,https://tourisme.example/evenements/exposition...
6,2025-12-18T10:35:57,https://tourisme.example/agenda.html,"Roméo et Juliette, le spectacle de danse Du 18...","Roméo et Juliette, le spectacle de danse",Du 18 décembre au 31 décembre,https://tourisme.example/evenements/romeo-juli...
7,2025-12-18T10:35:57,https://tourisme.example/agenda.html,Le sapin de verre de Noël de Bordeaux Du 18 dé...,Le sapin de verre de Noël de Bordeaux,Du 18 décembre au 04 janvier,https://tourisme.example/evenements/sapin-verr...
8,2025-12-18T10:35:57,https://tourisme.example/agenda.html,Le Noël de la Pépi ! Du 18 décembre au 21 déce...,Le Noël de la Pépi !,Du 18 décembre au 21 décembre,https://tourisme.example/evenements/noel-pepi....
9,2025-12-18T10:35:57,https://tourisme.example/agenda.html,Mapping / Projection de Noël sur le Grand Théâ...,Mapping / Projection de Noël sur le Grand Théâ...,Du 18 décembre au 04 janvier,https://tourisme.example/evenements/mapping-pr...
